In [5]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../src/')
import sample
from importlib import reload

In [17]:
reload(sample)

<module 'sample' from '../src/sample.py'>

In [18]:
funcs = sample.load_funcs('patient2/funcs_full.csv')
funcs

,Roi,FunctionType,DoseLevel,PercentVolume,EudParameterA,Weight
0,PTV 4/7/20,MinDose,"[20.0, 6270.0]",0,NaN,"[1.0, 200.0]"
1,PTV 4/7/20,MaxDose,"[6271.0, 7550.0]",0,NaN,1
2,Lungs,MaxDvh,"[0.0, 2000.0]","[35.0, 40.0]",NaN,1
3,SpinalCord (Thorax),MaxDose,"[0.0, 5000.0]",0,NaN,1
4,Esophagus,MaxDose,"[0.0, 6930.0]",0,NaN,1
5,Heart,MaxEud,"[0.0, 3500.0]",NaN,"[1.0, 20.0]",1
6,Blah,MinDvh,"[20.0, 50.0]","[10.0, 20.0]",NaN,1


In [19]:
pars = sample.init_pars(funcs)
pars

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,20.0,0.0,NaN,1.0
1,0,1,PTV 4/7/20,7550.0,0.0,NaN,1.0
2,0,2,Lungs,2000.0,40.0,NaN,1.0
3,0,3,SpinalCord (Thorax),5000.0,0.0,NaN,1.0
4,0,4,Esophagus,6930.0,0.0,NaN,1.0
5,0,5,Heart,3500.0,NaN,20.0,1.0
6,0,6,Blah,20.0,10.0,NaN,1.0


In [20]:
goals = sample.init_goals(funcs)
goals

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDose,AtLeast,20.0,0.0
1,PTV 4/7/20,MaxDose,AtMost,7550.0,0.0
2,Lungs,MaxDvh,AtMost,2000.0,40.0
3,SpinalCord (Thorax),MaxDose,AtMost,5000.0,0.0
4,Esophagus,MaxDose,AtMost,6930.0,0.0
5,Heart,MaxEud,AtMost,3500.0,20.0
6,Blah,MinDvh,AtLeast,20.0,10.0


In [21]:
results = sample.init_results(goals)
results

,Sample,Flag,0,1,2,3,4,5,6


In [23]:
temp = pd.read_pickle('patient2/pars_full.npy')
len(temp)

768

In [29]:
temp = pd.read_pickle('patient2/_full.npy')
len(temp)

365

In [32]:
6920/2

3460.0

# Functions

In [225]:
# Define using a dictionary
data = [{
    'Roi': 'PTV 4/7/20',
    'FunctionType': 'MinDose',
    'DoseLevel': 6270,
    'PercentVolume': 0,
    'Weight': 1, 
}, {
    'Roi': 'PTV 4/7/20',
    'FunctionType': 'MaxDose',
    'DoseLevel': [6271, 7550],
    'PercentVolume': 0,
    'Weight': 1, 
}, {
    'Roi': 'Lungs',
    'FunctionType': 'MaxDvh',
    'DoseLevel': [0, 2000],
    'PercentVolume': 35,
    'Weight': 1,
}, {
    'Roi': 'SpinalCord (Thorax)',
    'FunctionType': 'MaxDose',
    'DoseLevel': [0, 5000],
    'PercentVolume': 0,
    'Weight': 1,
}, {
    'Roi': 'Esophagus',
    'FunctionType': 'MaxDose',
    'DoseLevel': [0, 6930],
    'PercentVolume': 0,
    'Weight': 1
}, {
    'Roi': 'Heart',
    'FunctionType': 'MaxEud',
    'DoseLevel': [0, 3500],
    'EudParameterA': 1,
    'Weight': 1
}]
columns = ['Roi', 'FunctionType', 'DoseLevel', 'PercentVolume', 'EudParameterA', 'Weight']
funcs = pd.DataFrame(data=data, columns=columns)
funcs

,Roi,FunctionType,DoseLevel,PercentVolume,EudParameterA,Weight
0,PTV 4/7/20,MinDose,6270,0.0,NaN,1
1,PTV 4/7/20,MaxDose,"[6271, 7550]",0.0,NaN,1
2,Lungs,MaxDvh,"[0, 2000]",35.0,NaN,1
3,SpinalCord (Thorax),MaxDose,"[0, 5000]",0.0,NaN,1
4,Esophagus,MaxDose,"[0, 6930]",0.0,NaN,1
5,Heart,MaxEud,"[0, 3500]",NaN,1.0,1


In [246]:
# Define using a csv file
#
# Any entry that was a range of values got interpreted as a string,
# so I had to add some additional processing to get back the original
# list that I wanted.
def init_funcs(fpath):
    """Initialize constituent functions.
    
    Parameters
    ----------
    fpath : str
        File path to CSV file.
    
    Returns
    -------
    pandas.DataFrame
        Constituent function specifications.
    
    """
    funcs = pd.read_csv(fpath)
    funcs = funcs.astype(object)
    col_names = ['DoseLevel', 'PercentVolume', 'EudParameterA', 'Weight']
    for index, row in funcs.iterrows():
        for col in col_names:
            if isinstance(row[col], str):
                temp = [float(s) for s in re.findall(r'\d+\.\d+|\d+', row[col])]
                funcs.loc[index, col] = (temp, temp[0])[len(temp) == 1]
    return funcs

In [247]:
funcs2 = init_funcs('../src/patient2_funcs.csv')
funcs2

,Roi,FunctionType,DoseLevel,PercentVolume,EudParameterA,Weight
0,PTV 4/7/20,MinDose,6270,0,NaN,1
1,PTV 4/7/20,MaxDose,"[6271.0, 7550.0]",0,NaN,1
2,Lungs,MaxDvh,"[0.0, 2000.0]",35,NaN,1
3,SpinalCord (Thorax),MaxDose,"[0.0, 5000.0]",0,NaN,1
4,Esophagus,MaxDose,"[0.0, 6930.0]",0,NaN,1
5,Heart,MaxEud,"[0.0, 3500.0]",NaN,1,1


# Parameters

In [231]:
def init_pars(funcs):
    """Initialize constituent function parameters to maximum values.
    
    Parameters
    ----------
    funcs : pandas.DataFrame
        Constituent function specifications.
    
    Returns
    -------
    pandas.DataFrame
        Sampled constituent function parameters.
    
    """
    data = [{
        'Sample': 0,
        'Term': ii,
        'Roi': funcs.iloc[ii]['Roi'],
        'DoseLevel': np.max(funcs.iloc[ii]['DoseLevel']),
        'PercentVolume': np.max(funcs.iloc[ii]['PercentVolume']),
        'EudParameterA': np.max(funcs.iloc[ii]['EudParameterA']),
        'Weight': np.max(funcs.iloc[ii]['Weight'])
    } for ii in range(6)]
    columns = ['Sample', 'Term', 'Roi', 'DoseLevel',
               'PercentVolume', 'EudParameterA', 'Weight']
    return pd.DataFrame(data=data, columns=columns)

In [232]:
pars = init_pars(funcs)
pars

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,6270,0.0,NaN,1
1,0,1,PTV 4/7/20,7550,0.0,NaN,1
2,0,2,Lungs,2000,35.0,NaN,1
3,0,3,SpinalCord (Thorax),5000,0.0,NaN,1
4,0,4,Esophagus,6930,0.0,NaN,1
5,0,5,Heart,3500,NaN,1.0,1


In [233]:
pars2 = init_pars(funcs2)
pars2

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,6270.0,0.0,NaN,1
1,0,1,PTV 4/7/20,7550.0,0.0,NaN,1
2,0,2,Lungs,2000.0,35.0,NaN,1
3,0,3,SpinalCord (Thorax),5000.0,0.0,NaN,1
4,0,4,Esophagus,6930.0,0.0,NaN,1
5,0,5,Heart,3500.0,NaN,1.0,1


In [234]:
def sample_pars(funcs, pars):
    """Sample constituent function parameters.
    
    Parameters
    ----------
    funcs : pandas.DataFrame
        Constituent function specifications.
    pars : pandas.DataFrame
        Sampled constituent function parameters.
    
    Returns
    -------
    pandas.DataFrame
        Updated sampled constituent function parameters.
    
    """
    names = ['DoseLevel', 'PercentVolume', 'EudParameterA', 'Weight']
    sample = max(pars['Sample']) + 1
    new_pars = []
    for idx, row in funcs.iterrows():
        new_row = {'Sample': sample, 'Term': idx, 'Roi': row['Roi']}
        for ii in range(len(names)):
            if isinstance(row[names[ii]], list):
                new_row[names[ii]] = np.random.randint(*row[names[ii]])
            else:
                new_row[names[ii]] = row[names[ii]]
        new_pars.append(new_row)
    return pars.append(new_pars, ignore_index=True)

In [235]:
pars = sample_pars(funcs, pars)
pars

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,6270,0.0,NaN,1
1,0,1,PTV 4/7/20,7550,0.0,NaN,1
2,0,2,Lungs,2000,35.0,NaN,1
3,0,3,SpinalCord (Thorax),5000,0.0,NaN,1
4,0,4,Esophagus,6930,0.0,NaN,1
5,0,5,Heart,3500,NaN,1.0,1
6,1,0,PTV 4/7/20,6270,0.0,NaN,1
7,1,1,PTV 4/7/20,7380,0.0,NaN,1
8,1,2,Lungs,1762,35.0,NaN,1
9,1,3,SpinalCord (Thorax),3633,0.0,NaN,1


In [236]:
pars2 = sample_pars(funcs2, pars2)
pars2

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,6270.0,0.0,NaN,1
1,0,1,PTV 4/7/20,7550.0,0.0,NaN,1
2,0,2,Lungs,2000.0,35.0,NaN,1
3,0,3,SpinalCord (Thorax),5000.0,0.0,NaN,1
4,0,4,Esophagus,6930.0,0.0,NaN,1
5,0,5,Heart,3500.0,NaN,1.0,1
6,1,0,PTV 4/7/20,6270.0,0.0,NaN,1
7,1,1,PTV 4/7/20,6617.0,0.0,NaN,1
8,1,2,Lungs,1695.0,35.0,NaN,1
9,1,3,SpinalCord (Thorax),2963.0,0.0,NaN,1


# Goals

In [250]:
# Define based on constituent functions
def init_goals(funcs):
    """Initialize clinical goals based on constituent functions.
    
    Parameters
    ----------
    funcs : pandas.DataFrame
        Constituent function specifications.
    
    Returns
    -------
    pandas.DataFrame
        Clinical goal specifications.
    
    """
    data = [{
        'Roi': funcs.iloc[ii]['Roi'],
        'Type': funcs.iloc[ii]['FunctionType'],
        'GoalCriteria': ('AtLeast', 'AtMost')\
                        ['Max' in funcs.iloc[ii]['FunctionType']],
        'AcceptanceLevel': np.max(funcs.iloc[ii]['DoseLevel']),
        'ParameterValue': (funcs.iloc[ii]['PercentVolume'],
                           funcs.iloc[ii]['EudParameterA'])\
                          ['Eud' in funcs.iloc[ii]['FunctionType']]
    } for ii in range(6)]
    columns = ['Roi', 'Type', 'GoalCriteria', 'AcceptanceLevel', 'ParameterValue']
    return pd.DataFrame(data=data, columns=columns)

In [298]:
goals = init_goals(funcs)
goals

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDose,AtLeast,6270,0.0
1,PTV 4/7/20,MaxDose,AtMost,7550,0.0
2,Lungs,MaxDvh,AtMost,2000,35.0
3,SpinalCord (Thorax),MaxDose,AtMost,5000,0.0
4,Esophagus,MaxDose,AtMost,6930,0.0
5,Heart,MaxEud,AtMost,3500,1.0


In [47]:
# Modify goals
goals.loc[goals['GoalCriteria'] == 'AtLeast', ['Type', 'ParameterValue']] = ['MinDvh', 99]
goals.loc[goals['Roi'] == 'Heart', ['Type', 'ParameterValue']] = ['AverageDose', 0]
goals

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDvh,AtLeast,6270,99.0
1,PTV 4/7/20,MaxDose,AtMost,7550,0.0
2,Lungs,MaxDvh,AtMost,2000,35.0
3,SpinalCord (Thorax),MaxDose,AtMost,5000,0.0
4,Esophagus,MaxDose,AtMost,6930,0.0
5,Heart,AverageDose,AtMost,3500,0.0


In [239]:
goals2 = init_goals(funcs2)
goals2

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDose,AtLeast,6270.0,0.0
1,PTV 4/7/20,MaxDose,AtMost,7550.0,0.0
2,Lungs,MaxDvh,AtMost,2000.0,35.0
3,SpinalCord (Thorax),MaxDose,AtMost,5000.0,0.0
4,Esophagus,MaxDose,AtMost,6930.0,0.0
5,Heart,MaxEud,AtMost,3500.0,1.0


In [240]:
goals2.loc[goals2['GoalCriteria'] == 'AtLeast', ['Type', 'ParameterValue']] = ['MinDvh', 99]
goals2.loc[goals2['Roi'] == 'Heart', ['Type', 'ParameterValue']] = ['AverageDose', 0]
goals2

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDvh,AtLeast,6270.0,99.0
1,PTV 4/7/20,MaxDose,AtMost,7550.0,0.0
2,Lungs,MaxDvh,AtMost,2000.0,35.0
3,SpinalCord (Thorax),MaxDose,AtMost,5000.0,0.0
4,Esophagus,MaxDose,AtMost,6930.0,0.0
5,Heart,AverageDose,AtMost,3500.0,0.0


In [252]:
# Define from CSV
goals3 = pd.read_csv('../src/patient2_goals.csv')
goals3

,Roi,Type,GoalCriteria,AcceptanceLevel,ParameterValue
0,PTV 4/7/20,MinDose,AtLeast,6270,99
1,PTV 4/7/20,MaxDose,AtMost,7550,0
2,Lungs,MaxDvh,AtMost,2000,35
3,SpinalCord (Thorax),MaxDose,AtMost,5000,0
4,Esophagus,MaxDose,AtMost,6930,0
5,Heart,AverageDose,AtMost,3500,0


# Results

In [269]:
columns = ['Sample', 'Flag', 0, 1, 2, 3, 4]
results = pd.DataFrame(columns=columns)
results

,Sample,Flag,0,1,2,3,4


In [393]:
def init_results(goals):
    columns = ['Sample', 'Flag']
    columns += list(np.arange(len(goals)))
    return pd.DataFrame(columns=columns)


In [398]:
def init_stats():
    columns = ['Sample', 'Roi', 'Min', 'Average', 'Max', 'D99',
               'D98', 'D95', 'D90', 'D50', 'D10', 'D5', 'D2', 'D1']
    return pd.DataFrame(columns=columns)

In [433]:
x = [1, 3]
low = x[0]
high = x[1] + 1
np.random.randint(low, high)

1

In [425]:
def _get_sample(df):
    """Get current sample index."""
    if len(df) == 0:
        return 0
    else:
        return max(df['Sample']) + 1

,Sample,Flag,0,1,2,3,4


In [279]:
new_row = {'Sample': 0, 'Flag': 0}
for ii in range(len(goals)):
    new_row[ii] = 30
results.append(new_row, ignore_index=True)

,Sample,Flag,0,1,2,3,4,5
0,0,0,30,30,30,30,30,30.0


In [351]:
def _get_roi_result(dose, goal):
    """Get result for given clinical goal."""
    if 'Dose' in goal['Type']:
        dose = re.findall('[A-Z][^A-Z]*', goal['Type'])[0]
        return dose
    elif 'Dvh' in goal['Type']:
        return row['ParameterValue']
    else:
        return np.nan

In [365]:
for index, row in goals3.iterrows():
    print(_get_roi_result(index, row))

Min
Max
35
Max
Max
Average


In [371]:
def get_results(plan, flag, goals, results):
    """Get clinical goal results.

    Parameters
    ----------
    flag : int
        0 = success, 1 = normalization failed, 2 = optimization failed
    pandas.DataFrame
        Clinical goal specifications.

    Returns
    -------
    pandas.DataFrame
        Clinical goal results.

    """
    dose = 1
    sample = _get_sample(results)
    new_results = {'Sample': sample, 'Flag': flag}
    for index, row in goals.iterrows():
        result = _get_roi_result(index, row)
        print(type(result))
        new_results[index] = result
    return results.append(new_results, ignore_index=True)

In [372]:
columns = ['Sample', 'Flag', 0, 1, 2, 3, 4]
results = pd.DataFrame(columns=columns)
results = get_results(0, 0, goals2, results)
results

<class 'int'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>


,Sample,Flag,0,1,2,3,4,5
0,0,0,0,Max,0,Max,Max,Average


In [336]:
results.dtypes

Sample     object
Flag       object
0          object
1          object
2         float64
3          object
4          object
5          object
dtype: object

In [281]:
def get_results(flag, goals, results):
    """

    Parameters
    ----------
    flag : int
        0 = success, 1 = normalization failed, 2 = optimization failed
    pandas.DataFrame
        Clinical goal specifications.

    Returns
    -------
    pandas.DataFrame
        Clinical goal results.

    """
    sample = _get_sample(results)
    new_results = {'Sample': sample, 'Flag': flag}
    for idx, row in goals.iterrows():
        for ii in range(len(goals)):
            new_results[ii] = 10
    return results.append(new_results, ignore_index=True)

In [282]:
get_results(1, goals, results)

,Sample,Flag,0,1,2,3,4,5
0,0,1,10,10,10,10,10,10.0


In [436]:
pars = pd.read_pickle('pars_6_22.npy')
pars

,Sample,Term,Roi,DoseLevel,PercentVolume,EudParameterA,Weight
0,0,0,PTV 4/7/20,6270.0,0.0,NaN,1
1,0,1,PTV 4/7/20,7550.0,0.0,NaN,1
2,0,2,Lungs,2000.0,35.0,NaN,1
3,0,3,SpinalCord (Thorax),5000.0,0.0,NaN,1
4,0,4,Esophagus,6930.0,0.0,NaN,1
5,0,5,Heart,3500.0,NaN,1.0,1
6,1,0,PTV 4/7/20,6270.0,0.0,NaN,1
7,1,1,PTV 4/7/20,7314.0,0.0,NaN,1
8,1,2,Lungs,1512.0,35.0,NaN,1
9,1,3,SpinalCord (Thorax),2259.0,0.0,NaN,1


In [437]:
results = pd.read_pickle('results_6_22.npy')
results

,Sample,Flag,0,1,2,3,4,5
0,0.0,0.0,6270.000270,8579.848633,2243.763593,5760.807129,7824.008789,2808.294699
1,3.0,0.0,6270.000179,36306.218750,6269.832601,14634.908203,9653.461914,3671.885750


In [439]:
stats = pd.read_pickle('stats_6_22.npy')
stats

,Sample,Roi,Min,Average,Max,D99,D98,D95,D90,D50,D10,D5,D2,D1
0,0,Heart,113.045166,2808.294699,8328.268555,159.168080,178.354093,226.360112,303.659702,2517.188416,6040.023657,6895.808142,7364.301482,7549.880795
1,0,PTV 4/7/20,4764.008789,7568.895593,8579.848633,6270.000270,6450.611917,6709.611323,6909.803082,7607.642901,8183.720053,8275.887616,8352.290161,8392.111863
2,0,Esophagus,175.875900,3844.298444,7824.008789,199.834958,211.393508,237.924117,278.755871,3933.336119,7526.400582,7567.836137,7607.753331,7638.289091
3,0,SpinalCord (Thorax),0.001909,1827.747567,5760.807129,0.001909,0.001909,0.001909,3.861012,682.988621,5049.693774,5304.265394,5458.225047,5530.640992
4,0,Lungs,58.716351,2633.600471,8358.260742,154.083038,192.137214,285.435613,456.703415,1584.941437,6647.220245,7138.822608,7473.808622,7640.188465
5,3,Heart,139.044128,3671.885750,27299.261719,232.643967,271.410906,369.650924,591.100564,2856.940281,7794.665759,10383.336258,13082.117060,15236.950198
6,3,PTV 4/7/20,3308.770508,19856.035547,36306.218750,6270.000179,7067.612377,8815.679111,10971.467598,20675.108615,26702.119635,28175.222811,29870.298436,31076.881060
7,3,Esophagus,345.628662,3741.611244,9653.461914,399.190809,416.636344,459.630435,533.355727,4667.186123,6621.070694,7143.238327,7783.210053,8155.349601
8,3,SpinalCord (Thorax),0.003441,3600.854908,14634.908203,0.003441,0.003441,0.003441,1.386972,1955.499833,8492.623042,9155.431686,9879.519533,10383.554288
9,3,Lungs,141.802216,6806.169909,33780.300781,372.370815,474.586102,731.851244,1146.068941,3949.866861,17462.291105,20195.613699,22964.995976,24764.334547
